<a href="https://colab.research.google.com/github/marypthomas/ai-bootcamp-osu/blob/main/Session_9/AIBootCamp_NaiveBayes4SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI Bootcamp: Naive Bayes**
**Author**:
- Prof. Eric Fosler-Lussier, The Ohio State University

## Goals
The goals of this notebook are to familarize you with:
*   Naive Bayes
*   Binary Classification
*   Data exploration

### **Part 0: Initial setup**

**0.1:** While not completely necessary for this assignment, you may want to familiarize yourself with the following packages: [numpy](https://numpy.org), [scikit-learn](https://scikit-learn.org), [pandas](https://pandas.pydata.org), [matplotlib](https://matplotlib.org).

---
---

**Part 1: A Simple Bayes Net: Naive Bayes**

In the lecture, we discussed how conditional independences of a joint probablity distribution get encoded by a Bayesian Network (BN). One of the simplest form of BNs is the Naive Bayes (NB) model which encodes a set of simple conditional independences: 

- Given a single cause all of the effects are independent from each other.
- Mathematically: 
$P($*cause*$, $*effect*$_1, ..., $*effect*$_n) = P($*cause*$) \prod_i P($*effect*$_i|$*cause*$)$ 

NB can be used for classification by assuming that cause is the true (unknown) label and it (probabilistically) generates all of the features (effects) while features are independent given the cause. 

For example, in sentiment analysis the *cause* is the author's sentiment (say, unknown label from the set of {sad, happy, feared, suprised, disgusted, angry}) and the *effects* are words that s/he writes. The simplifying assumption of NB says that knowing the latent sentiment, words of the sentence are independent. We know this assumption is not true because grammar and word-use impose some dependency structure between words in the sentence, but we choose to ignore that in this model.

Although simple, NB has shown good performance in many classifcation tasks and has become a standard classic baseline for classification. 

Today we want to perform Twitter sentiment analysis using NB. The goal is to figure out if a tweet has a positive or negative sentiment about the weather.  

**1.0:** Set up the environment (you can click on the play button below to import the appropriate modules).

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

**1.1** Read the data from GitHub into a pandas dataframe.

In [ ]:
TweetUrl='https://github.com/efosler/cse5522data/raw/master/db3_final_clean.csv'
tweet_dataframe=pd.read_csv(TweetUrl)

**1.2** Print out the top of the dataframe to make sure that the data loaded correctly.  It should be a data table with three columns (weight, tweet, label), and 3697 rows.

In [ ]:
display(tweet_dataframe.shape)
tweet_dataframe.head()

Labels are -1 and +1 for negative and positive sentiments respectively. Multiple judges have been asked to choose a label for a tweet (this is an example of crowd-sourcing) from five possible labels: 

- Tweet is not relevant to weather. 
- I can't tell the sentiment. 
- Neutral: author just sharing information. 
- Positive
- Negative

The majority vote was picked as the label and its ratio was set as the weight of the tweet. So for the tweet in row 2 above, 61% of judges voted that the label is negative.

Note that tweets have been pre-processed (or cleaned). For example, :) and :( :) were replaced with "sad" and "smiley" and numbers with "num", etc. You can go further (as we ask in 1.12) and remove the stop words, i.e., repetitive non-informative words such as am, is, and are. 

**1.3.** In the next step, we should build our feature matrix by converting the string of words to a vector of numeric values. 

First we need to assign a unique id to each word and create the feature matrix with correct size:

In [ ]:
# wordDict maps words to id
# X is the document-word matrix holding the presence/absence of words in each tweet
wordDict = {}
idCounter = 0
for i in range(tweet_dataframe.shape[0]):
  allWords = tweet_dataframe.iloc[i,1].split(" ")
  for word in allWords:
    if word not in wordDict:
      wordDict[word] = idCounter
      idCounter += 1
X = np.zeros((tweet_dataframe.shape[0], idCounter),dtype='float')

Checking head of the dictionary:

In [ ]:
dict(list(wordDict.items())[0:10])

**1.4:** The simplest way of coding a tweet to numbers is to mark the occurrence of a word, and forget about its frequency in the document (tweet). This works well with tweets as there are not many repetitive words in a single tweet. So let's fill the document-word matrix:

In [ ]:
for i in range(tweet_dataframe.shape[0]):
  allWords = tweet_dataframe.iloc[i,1].split(" ")
  for word in allWords:
    X[i, wordDict[word]]  = 1

Now we check if the number of words are correct:

In [ ]:
np.sum(X[0:5, ], axis = 1)

Finally, we extract the labels from the dataframe:

In [ ]:
y = np.array(tweet_dataframe.iloc[:,2])
y[0:5]

Let's compute the total number of positive and negative tweets:

In [ ]:
numNeg = np.where(y > 0)[0][0] - 1
numPos = len(y) - numNeg
probNeg = numNeg / (numNeg + numPos)
probPos = 1 - probNeg
display(numNeg, numPos, probNeg, probPos)

So samples 0:1649 are negative and 1650:-1 are positive.

**1.5: Train/Test Split** Now with do the 20/80 split and learn the word probabilities using the 80 % part and test the NB performance on the 20 % part. 

In [ ]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 0)
display(xTrain.shape, xTest.shape, yTrain.shape, yTest.shape)

**1.6: Computing Probabilities by Counting** Now the real work begins. Write the code that, from the train feature matrix xTrain computes the needed word probabilites, i.e., $P(word|label)$ where label is + or - and word is any of the words saved in the `wordDict`:

In [ ]:
# compute three distributions (four variables):
#
# probWordGivenPositive: P(word|Sentiment = +ive)
# probWordGivenNegative: P(word|Sentiment = -ive)
# priorPositive: P(Sentiment = +ive)
# priorNegative: P(Sentiment = -ive)
#  (note these last two form one distribution)

# compute distributions here - to compute P(word|Sentiment = +ive), select all positive tweets,
#     then count the number of times each word occurs.  Normalize by total number of words in positive tweets.
# do the same thing for negative tweets.


# checking the results
display(probWordGivenPositive[0:5])
display(probWordGivenNegative[0:5])
display(priorPositive, priorNegative)

Note that you only needed to compute $P(word = 1| +)$ or $P(word = 1| -)$ and the probabilities of the word being absent from a tweet is just 1 minus those probabilities. 

My answer is hidden below.

In [ ]:
# set up count vectors fpr
probWordGivenPositive = np.zeros(len(wordDict), dtype='float') #X.shape[1] == len(wordDict)
probWordGivenNegative = np.zeros(len(wordDict), dtype='float')
priorPositive = 0
priorNegative = 0

# note that this works because xTrain only has 1 if the word is present, not the count.
probWordGivenPositive = sum(xTrain[yTrain == +1,]) / sum(yTrain == +1)
probWordGivenNegative = sum(xTrain[yTrain == -1,]) / sum(yTrain == -1)

#### EDIT FOR NEXT SECTION:
#### note that the if probWordGivenPositive has a zero prob, then the log(prob) is undefined
#### so we can fix this by adding a small epsilon to both Count(word=1|+ive) and Count(word=0|-ive)
####
#### this idea is called Laplace smoothing
epsilon=0.001
probWordGivenPositive = (sum(xTrain[yTrain == +1,])+epsilon) / (sum(yTrain == +1)+2*epsilon)
probWordGivenNegative = (sum(xTrain[yTrain == -1,])+epsilon) / (sum(yTrain == -1)+2*epsilon)

priorNegative = sum(yTrain == -1) / len(yTrain)
priorPositive = sum(yTrain == +1) / len(yTrain)

# checking the results
display(probWordGivenPositive[0:5])
display(probWordGivenNegative[0:5])
display(priorPositive, priorNegative)

However, as we see in 1.6, for convenience, we will also want to compute $log P(word = 1 | +)$, $log P(word = 0 | +)$, $log P(word = 1 | -)$ and $log P(word = 0 | -)$.  Also we should compute the log priors.  Let's do so now.

In [ ]:
# compute the following:
# logProbWordPresentGivenPositive
# logProbWordAbsentGivenPositive
# logProbWordPresentGivenNegative
# logProbWordAbsentGivenNegative
# logPriorPositive
# logPriorNegative


# Did this work, or did you get an error?  (Read below.)

You likely received an error when you tried to take $log(0)$ at some point.  Can your group think of a way to avoid taking $log(0)$?  Check in with your instructor/TA to see if what you're thinking will work.  Implement that change in your code above.

In [ ]:
logProbWordPresentGivenPositive = np.log(probWordGivenPositive)
logProbWordAbsentGivenPositive = np.log(1-probWordGivenPositive)
logProbWordPresentGivenNegative = np.log(probWordGivenNegative)
logProbWordAbsentGivenNegative = np.log(1-probWordGivenNegative)
logPriorPositive = np.log(priorPositive)
logPriorNegative = np.log(priorNegative)

**1.6: Math of NB** Here we provide the derivation of NB when we want to classify the $i$th tweet $\textbf{x}^{(i)}$ and the size of dictionary is $p$, i.e., each tweet is a binary vector of size $p$ as $\textbf{x}^{(i)} = (x_1^{(i)},\dots, x_p^{(i)})$. 

Note that we computed $P(x_j^{(i)} = 1|+)$ and $P(x_j^{(i)} = 1|-)$ in above code from `xTrain` and now want to classify `xTest` samples.

**Classification Rule:** For each tweet $i$ NB classifier assigns label + if $P(+|\textbf{x}^{(i)}) > P(-|\textbf{x}^{(i)})$ and negative otherwise. 

These posterior probabilities can be computed using prior probabilities (that we got from `xTrain`) and Bayes rule as follows: 

\begin{align}
P(+|\textbf{x}^{(i)}) &= \alpha P(\{\textbf{x}^{(i)}\}_{i=1}^n | +)P(+) 
\\
(\text{NB Assumption}) &= \alpha P(+) \prod_{j=1}^p P(x_j^{(i)}|+)
\end{align}

For computational convinence (preventing underflow while dealing with small numbers) we work with the $\log$ of probabilities:

\begin{align} 
\log(P(+|\textbf{x}^{(i)})) &\propto \log P(+) + \sum_{j=1}^p \log P(x_j^{(i)}|+) 
\\
\log(P(-|\textbf{x}^{(i)})) &\propto \log P(-) + \sum_{j=1}^p \log P(x_j^{(i)}|-) 
\end{align} 

Finally we can compute the confidence of our prediction as the log of the ratio of posteriors: 
$\log(\frac{P(\text{predicted label}|\textbf{x}^{(i)})}{P(\text{the other label}|\textbf{x}^{(i)})})$


**1.7: Implementing NB** Now write a function that takes a row of `xTest` and output a label for it based on NB classification rule. 


In [ ]:
# classifyNB: 
#   words - vector of words of the tweet (binary vector)
#   logProbWordPresentGivenPositive - log P(x_j = 1|+)
#   logProbWordAbsentGivenPositive  - log P(x_j = 0|+)
#   logProbWordPresentGivenNegative - log P(x_j = 1|-)
#   logProbWordAbsentGivenNegative  - log P(x_j = 0|-)
#   logPriorPositive - log P(+)
#   logPriorNegative - log P(-)
#   returns (label of x according to the NB classification rule, confidence about the label)

# Note: you can also change the function definition if you wish to encapsulate all six log probs
# as one model; just make sure to follow through below

def classifyNB(words,logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
               logPriorPositive, logPriorNegative):
  # fill in function definition here
  
print(classifyNB(xTest[700, ], logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
               logPriorPositive, logPriorNegative)))

In [ ]:
def classifyNB(words,logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
               logPriorPositive, logPriorNegative):
    positivePosterior=logPriorPositive + sum(logProbWordPresentGivenPositive[words==1]) + sum(logProbWordAbsentGivenPositive[words==0])
    negativePosterior=logPriorNegative + sum(logProbWordPresentGivenNegative[words==1]) + sum(logProbWordAbsentGivenNegative[words==0])

    if (positivePosterior > negativePosterior):
        return (+1, positivePosterior-negativePosterior)
    else:
        return (-1, negativePosterior-positivePosterior)

print((classifyNB(xTest[700, ], logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
               logPriorPositive, logPriorNegative)),yTest[700])

**1.8:** Compute the output of `classifyNB` for all test data and output the average error.  

In [ ]:
# testNB: Classify all xTest
#   xTest - test data features
#   yTest - true label of test data
#   logProbWordPresentGivenPositive - log P(x_j = 1|+)
#   logProbWordAbsentGivenPositive  - log P(x_j = 0|+)
#   logProbWordPresentGivenNegative - log P(x_j = 1|-)
#   logProbWordAbsentGivenNegative  - log P(x_j = 0|-)
#   logPriorPositive - log P(+)
#   logPriorNegative - log P(-)
#   returns Average test error
def testNB(xTest, yTest, 
           logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
           logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
           logPriorPositive, logPriorNegative):
    correct=0
    for i in range(len(yTest)):
        (y,diff)=classifyNB(xTest[i, ], logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
                            logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
                            logPriorPositive, logPriorNegative)
        if y==yTest[i]:
            correct=correct+1
            
    avgErr=1-(float(correct)/float(len(yTest)))
  
    print("Average error of NB is", avgErr)
    return avgErr

testNB(xTest, yTest, 
       logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
       logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
       logPriorPositive, logPriorNegative)

**1.9: Ignoring absent words** The basic formalism asks you to take account of both present and absent words.  What happens if you ignore the absent words?  Does prediction change?